<a id="top"></a>

# Db2 11.5 External Tables
Updated: 2019-10-15

### External Tables
Db2 11.5 introduces support for External tables. External tables are objects that reside on a local or remote file system that Db2 can create or read as if they were standard SQL tables. The advantage of using external tables for Extraction-Transformation-Loading (ETL) processes is that they can be carried out using plain SQL. Because an SQL-based ETL process can be initiated from any SQL client, it eliminates the need for special ETL tools.

Use an external table to: 
* Load data from an external file into a table in the database
* Unload data from the database into an external file
* Store data outside the database while retaining the ability to query that data

External Table Usage:
* Use a FROM clause in a SELECT SQL statement as you would for any other table
* Specify the external table as the target table in one of the following SQL statements:
  - INSERT SQL
  - SELECT INTO SQL
  - CREATE EXTERNAL TABLE AS SELECT SQL

* You can perform operations such as casts, joins, and dropping columns to manipulate data during loading


### Load Db2 Extensions and Connect to the Database
The `connection` notebook contains the `CONNECT` statement which allows access to the `SAMPLE` database. If you need to modify the connection information, edit the `connection.ipynb` notebook.

In [ ]:
%run ../db2.ipynb
%run ../connection.ipynb

Retrieving the name of the notebook user for future use.

In [ ]:
current_user_rs = %sql -r VALUES CURRENT USER
schema = current_user_rs[1][0] # 'DB2USER1'
print("Connected user is: " + schema)

### External Table Path
In order for Db2 to access external tables, it must be given the relative access path for these objects. The next cell will get the current path that we are running in. This path will then be used to update the Database `EXTBL_LOCATION` field so that the reads and writes to external tables will run.

In [ ]:
import os
notebook_directory = os.path.abspath('.')
print('The current notebook directory is ' + notebook_directory)

To allow simultaneous users to access the external table directory, we are going to use `DB2INST1` home directory as the location of all of the files. 

In [ ]:
db2inst1_directory = '/home/db2inst1'

We create a string that will update the database configuration so that the `DB2INST1` directory will be used as the directory for all files.

In [ ]:
admin_cmd = 'UPDATE DB CONFIG FOR SAMPLE USING EXTBL_LOCATION "' + db2inst1_directory + '" IMMEDIATE'

The following SQL will use the Db2 `ADMIN_CMD` to update the database setting to allow for reading and writing of external files to this directory.

In [ ]:
%sql CALL ADMIN_CMD({admin_cmd})

### External Table Files
The three files that are used for the external table examples will be copied from the notebook directory to the `DB2INST1` home directory. If copies of the files already exist, they will not be replaced. If you do want to replace the existing files, remove the `-u` flag from the command.

In [ ]:
cmd = "cp -uf {0}/*.csv {1}".format(notebook_directory, db2inst1_directory)
print(cmd)
!{cmd}

### Stocks Tables
The following examples will use two files to demonstrate the use of External tables. The two files are:
- Stock Data - Information for all opening and closing values for the DJIA average in 2018
- Stock Labels - Full name of the stock indexes
The files are located within this notebook directory so when refering to these files in Db2 we must supply an absolute address, not a relative one to Db2.

### Absolute Path
We need to use the absolute path of a file for use with Db2, so we create the full paths for the two files.

In [ ]:
import os
djia_2018    = "{0}/{1}".format(db2inst1_directory,"djia-2018.csv")
djia_symbols = "{0}/{1}".format(db2inst1_directory,"djia-symbols.csv")
print(djia_2018)
print(djia_symbols)

The first step is to verify the files and display the first 5 lines of each file.

In [ ]:
import csv
from itertools import islice

print("Stock Values")
print("------------")
with open(djia_symbols) as csvfile:
    reader = csv.DictReader(csvfile)
    for row in islice(reader, 5): # first 5 only
        print(row['Symbol'], row['Company Name'])

print("")
print("Stock Symbols")
print("-------------")
with open(djia_2018) as csvfile:
    reader = csv.DictReader(csvfile)
    for row in islice(reader, 5): # first 5 only
        print(row['Stock'], row['Date'], row['High'], row['Low'], row['Open'], row['Close'], row['Volume'])

### Using SELECT (Transient) statements against External Tables
The following SQL will read the data from the `SYMBOLS` table and display it in a grid.

In [ ]:
%%sql -grid
SELECT * FROM EXTERNAL :djia_symbols (
    SYMBOL VARCHAR(10),
    NAME   VARCHAR(255)
    )
    USING ( DELIMITER ',' STRING_DELIMITER DOUBLE SKIP_ROWS 1 NOLOG TRUE )

This next SQL statement will retrieve all of the data from the 2018 stock transactions.

In [ ]:
%%sql
SELECT * FROM EXTERNAL :djia_2018 (
    SYMBOL    VARCHAR(4),
    TX_DATE   DATE,
    HIGH_VAL  DEC(9,2),
    LOW_VAL   DEC(9,2),
    OPEN_VAL  DEC(9,2),
    CLOSE_VAL DEC(9,2),
    VOLUME    INT
    )
    USING ( DELIMITER ',' STRING_DELIMITER DOUBLE SKIP_ROWS 1 NOLOG TRUE )

### Cataloging External Tables
The previous examples used a transient form of external table. The external table definition only existed during the execution of the SQL statement. Rather than define the external table every time, you have the option of cataloging the external table. When the external table is cataloged, you refer to it in the same way as a standard SQL table. The underlying file is checked at execution time, so it is possible that this file changes from day to day. Only when the SQL is executed is the existence of the file checked.

In [ ]:
%%sql 
DROP TABLE DJIA_2018;
DROP TABLE DJIA_SYMBOLS;

The `CREATE EXTERNAL TABLE` uses the same syntax as the transient example shown earlier. The only difference is that we are creating an actual entry in the Db2 catalog that will refer to this external table.

In [ ]:
%%sql
CREATE EXTERNAL TABLE DJIA_2018  
    (
    SYMBOL    VARCHAR(4),
    TX_DATE   DATE,
    HIGH_VAL  DEC(9,2),
    LOW_VAL   DEC(9,2),
    OPEN_VAL  DEC(9,2),
    CLOSE_VAL DEC(9,2),
    VOLUME    INT
    )
    USING ( DELIMITER ',' STRING_DELIMITER DOUBLE SKIP_ROWS 1 FILE_NAME :djia_2018 NOLOG TRUE );

CREATE EXTERNAL TABLE DJIA_SYMBOLS  
    (
    SYMBOL VARCHAR(4),
    NAME   VARCHAR(255)
    )
    USING ( DELIMITER ',' STRING_DELIMITER DOUBLE SKIP_ROWS 1 FILE_NAME :djia_symbols NOLOG TRUE ) ;

The external tables can now be referenced in a SQL statement in the same way as any other SQL table. External tables cannot be indexed or be part of summary tables. 

The next SQL statement retrieves only the transactions that exceeded 180 in value and belonged to AAPL.

In [ ]:
%sql -grid select * from djia_2018 where symbol='AAPL' and close_val > 180

Since these external tables are cataloged and look like any other SQL table, we can include joins and other calculations on the columns.

In [ ]:
%%sql -grid
SELECT S.NAME, MAX(HIGH_VAL) AS HIGH, MIN(LOW_VAL) AS LOW, MAX(VOLUME) AS MAX_VOLUME, MIN(VOLUME) AS MIN_VOLUME
  FROM DJIA_SYMBOLS S, DJIA_2018 T
WHERE
  S.SYMBOL = T.SYMBOL
GROUP BY S.NAME

## Additional External Table Scenarios
This section will explore some additional ways of using External tables. The `product_txs.csv` file contains approximately 100000 transactions that will be queried in this section. 

The first step is to get the absolute file location of the product file.

In [ ]:
import os
product_txs    = "{0}/{1}".format(db2inst1_directory,"product_txs.csv")

We check to make sure that we have access to the external file with the following SQL statement. Note that we can restrict the number of records being returned either with the `USING MAXROWS` clause of the external table statement, or by using the SQL `FETCH FIRST` syntax.

In [ ]:
%%sql -grid
SELECT * FROM EXTERNAL :product_txs
  ( 
  RECEIPT_ID BIGINT, 
  TX_DATE    CHAR(10),
  TX_NO      INT,
  PRODUCT_ID INT,
  PRODUCT    VARCHAR(100), 
  QUANTITY   INT ,
  ITEM_COST  DEC(15,2)
  ) 
  USING ( DELIMITER ',' SKIPROWS 1 NOLOG TRUE ) 
FETCH FIRST 100 ROWS ONLY

To make our life easier, we will create a catalog entry for this external table.

In [ ]:
%%sql  
DROP TABLE PRODUCT_TXS;
CREATE EXTERNAL TABLE PRODUCT_TXS
  ( 
  RECEIPT_ID BIGINT, 
  TX_DATE    CHAR(10),
  TX_NO      INT,
  PRODUCT_ID INT,
  PRODUCT    VARCHAR(100), 
  QUANTITY   INT ,
  ITEM_COST  DEC(15,2)
  ) 
  USING ( DATAOBJECT :product_txs DELIMITER ',' SKIPROWS 1 NOLOG TRUE )  

We should now be able to retrieve data from this table using standard SQL syntax.

In [ ]:
%sql SELECT * FROM PRODUCT_TXS FETCH FIRST 10 ROWS ONLY

For reference, let's see how many records are found in the product table.

In [ ]:
result = %sql -r -nq SELECT COUNT(*) FROM PRODUCT_TXS
product_count = result[1][0]
print(product_count)

## Merging Data From an External Table into a Database Table
The following SQL will merge data from the external table into an existing table. One shortcut that you can use with external tables is through the use of the `LIKE` clause. If a table already exists that has the same columns as the external table, you can use its definition rather than explicitly typing them out.

The real table that we are going to use is called `ASSOC_TRANS` and will be used as a template for our external table.

In [ ]:
%%sql 
DROP TABLE ASSOC_TRANS;
CREATE TABLE ASSOC_TRANS
  ( 
  RECEIPT_ID BIGINT, 
  TX_DATE    CHAR(10),
  TX_NO      INT,
  PRODUCT_ID INT,
  PRODUCT    VARCHAR(100), 
  QUANTITY   INT ,
  ITEM_COST  DEC(15,2)
  ) 
NOT LOGGED INITIALLY

The transactions in the external table that are not `Rake` are inserted into the `ASSOC_TRANS` table.

In [ ]:
%%sql
MERGE INTO ASSOC_TRANS T_OLD 
USING (
      SELECT * FROM EXTERNAL :product_txs
         LIKE ASSOC_TRANS USING ( DELIMITER ',' SKIPROWS 1 NOLOG TRUE )
      WHERE PRODUCT <> 'Rake'
      ) AS T_NEW 
ON (T_OLD.RECEIPT_ID = T_NEW.RECEIPT_ID and T_OLD.TX_NO = T_NEW.TX_NO) 
WHEN NOT MATCHED THEN 
  INSERT (RECEIPT_ID, TX_DATE, TX_NO, PRODUCT_ID, PRODUCT, QUANTITY, ITEM_COST) 
    VALUES (T_NEW.RECEIPT_ID, T_NEW.TX_DATE, T_NEW.TX_NO, T_NEW.PRODUCT_ID, T_NEW.PRODUCT,
            T_NEW.QUANTITY, T_NEW.ITEM_COST)

At this point the table should contain all records except the ones that were for `Rake`. We do a quick count to see how many records there are.

In [ ]:
%sql SELECT COUNT(*) FROM ASSOC_TRANS

The next SQL statement will merge all of the records from the external table into this regular SQL table. If the record already exists, it will be ignored, otherwise it will be inserted into the table.

In [ ]:
%%sql 
MERGE INTO ASSOC_TRANS T_OLD 
USING (
      SELECT * FROM EXTERNAL :product_txs 
         LIKE ASSOC_TRANS USING ( DELIMITER ',' SKIPROWS 1 NOLOG TRUE )
      ) AS T_NEW 
ON (T_OLD.RECEIPT_ID = T_NEW.RECEIPT_ID and T_OLD.TX_NO = T_NEW.TX_NO) 
WHEN NOT MATCHED THEN 
  INSERT (RECEIPT_ID, TX_DATE, TX_NO, PRODUCT_ID, PRODUCT, QUANTITY, ITEM_COST) 
    VALUES (T_NEW.RECEIPT_ID, T_NEW.TX_DATE, T_NEW.TX_NO, T_NEW.PRODUCT_ID, T_NEW.PRODUCT,
            T_NEW.QUANTITY, T_NEW.ITEM_COST)
ELSE IGNORE

Now let's find out how many records there are in the `ASSOC_TRANS` table and compare that to the original size of the external product table.

In [ ]:
%sql SELECT COUNT(*) FROM ASSOC_TRANS;

The product count should be the same.

In [ ]:
print(product_count)

## Creating an External Table
You can create external tables by using the `CREATE EXTERNAL TABLE ... AS SELECT...` syntax. The SELECT can contain any arbitrary SQL so the statements can become very complex. You must ensure that Db2 has write access to the directory where the file will be written out to.

First we must define the name of the external table that we want to write to. In order to do that, we generate a path that includes the current directory and append the schema (owner) and the name of the file to it.

In [ ]:
dumpfile = "{0}/{1}_dump.csv".format(db2inst1_directory,schema)
print(dumpfile)

The next SQL statement will output the contents of the `ASSOC_TRANS` table to the external file. You can limit the amount of data that gets written with the `USING` clause, or you can use SQL options to limit it.

In [ ]:
%%sql 
CREATE EXTERNAL TABLE :dumpfile 
  USING ( DELIMITER '|' NOLOG TRUE ) 
  AS SELECT * FROM ASSOC_TRANS FETCH FIRST 100 ROWS ONLY

You must make sure you have the proper permissions to read the file if using anything outside of the Db2 environment. The file that was generated was created under the `CONNECT` userid for this notebook. If you used `DB2INST1` then only that user can access the file. The following code will fail if the current user is not authorized to read the file.

In [ ]:
try:
    file  = open(dumpfile)
    file_contents = file.read(100)
    print(file_contents)
    file.close()
except Exception as e:
    print("Open failed")
    print(e)

However, as a SQL user, you can read it!

In [ ]:
%%sql
SELECT * FROM EXTERNAL :dumpfile LIKE ASSOC_TRANS USING ( DELIMITER '|' SKIPROWS 1 NOLOG TRUE ) 

[Back to Top](#top)
<a id='function'></a>

#### Credits: IBM 2019, George Baklarz [baklarz@ca.ibm.com], Phil Downey [phil.downey1@ibm.com]